# Preparation
First download the t7 file from http://cs.nott.ac.uk/~psxasj/download.php?file=vrn-unguided.t7

I use https://github.com/bshillingford/python-torchfile to parse the torch file.

In [2]:
!pip install torchfile

  Stored in directory: C:\Users\ellio\AppData\Local\pip\Cache\wheels\b1\c3\d6\9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.insert(0, '../python-torchfile')
import torchfile

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
t = torchfile.load('vrn-unguided.t7')

FileNotFoundError: [Errno 2] No such file or directory: 'vrn-unguided.t7'

## Exploring the torch file

In [5]:
ln = 0
list_file = open("list.txt", "w")

def lst(tobj, indent, f):
    global ln
    for module in tobj['modules']:
        f.write("{}\t{}{}\t{}\n".format(ln,
                                        "    " * indent,
                                        module._typename,
                                        module['weight'].shape 
                                            if 'weight' in module.__dir__() 
                                            else ''))
        ln += 1
        if 'modules' in module.__dir__():
            lst(module, indent + 1, f)

lst(t, 0, list_file)
list_file.close()

Click [list.txt](./list.txt) to see the list.

## Compare torch layer to keras layer
I want to be able to reference each layer by its line number.

In [8]:
def getr(tobj, ln):
    for module in tobj['modules']:
        if ln == 0:
            return module, 0
        ln -= 1
        if 'modules' in module.__dir__():
            got, ln = getr(module, ln)
            if got is not None:
                return got, 0

    return None, ln

def get(tobj, ln):
    tout, _ = getr(tobj, ln)
    return tout

def info(module):
    print("{}".format(module._typename))
    for prop in module.__dir__():
        value = module[prop]
        if value.__class__ == np.ndarray:
            value = value.shape
        if value.__class__ == list and len(value) > 0:
            value = value[0].shape
        print("{}\t{}".format(prop, value))

### First Conv layer

In [12]:
info(get(t, 0))

nn.SpatialConvolution
weight	(64, 3, 7, 7)
nOutputPlane	64
gradWeight	(64, 3, 7, 7)
bias	(64,)
nInputPlane	3
padH	3
padW	3
fgradInput	(0,)
_type	torch.FloatTensor
finput	(1, 147, 9216)
fmode	1
kH	7
bwmode	1
train	True
kW	7
groups	1
gradBias	(64,)
dW	2
dH	2
gradInput	(0,)
bdmode	1
output	(1, 64, 96, 96)
torch_typename	None


### First BatchNorm layer

In [13]:
info(get(t, 1))

nn.SpatialBatchNormalization
save_mean	(64,)
_type	torch.FloatTensor
running_mean	(64,)
weight	(64,)
train	True
affine	True
gradInput	(0,)
eps	1e-05
running_var	(64,)
gradWeight	(64,)
bias	(64,)
gradBias	(64,)
output	(1, 64, 96, 96)
save_std	(64,)
momentum	0.1
torch_typename	None


### Note
As I understand it, in Keras, when you run model.predict(), the Batch Norm layers use the saved running_mean and running_var values to normalize the data.<br>
This is how I first tried to run this model, but it didn't work!<br>
Finally, after a good amount of troubleshooting, I realized that the model would work if the Batch Norm layers were in train mode. Meaning, the mean and variance are updated each time.<br>
So, I set training=True for each BN layer. I set momentum=1 to keep results consistent.

# Build model in Keras
[vrn_torch_to_keras.py](./vrn_torch_to_keras.py) is the script that parses vrn-unguided.t7, builds the Keras model, copies the weights, and then writes the h5 file.